In [61]:
!pip install xgboost scikit-learn joblib pandas numpy fastapi uvicorn pyngrok python-multipart

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = '/content/drive/MyDrive/Colab Notebooks/Laptop_price.csv'  # Убедитесь, что путь к файлу верный
df = pd.read_csv(file_path)
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib

# Определите числовые и категориальные признаки
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

# Трансформеры для числовых и категориальных данных
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Комбинируем трансформеры
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])

# Создаем пайплайн
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])

# Обучаем модель
pipeline.fit(X_train, y_train)

# Сохраняем модель
joblib.dump(pipeline, '/content/drive/MyDrive/laptop_price_model.pkl')

['/content/drive/MyDrive/laptop_price_model.pkl']

In [50]:
!git branch -m main
!git config --global user.email "supertestuser1230@mail.ru"
!git config --global user.name "supertestuser1230"
!git init

fatal: not a git repository (or any of the parent directories): .git
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [51]:
!git add /content/drive/MyDrive/laptop_price_model.pkl
!git add /content/drive/MyDrive/Colab\ Notebooks/Laptop_price.csv
!git add /content/drive/MyDrive/Colab\ Notebooks/Pipeline.ipynb

In [52]:
!git commit -m "Добавлен ML-пайплайн"

[master (root-commit) 416490a] Добавлен ML-пайплайн
 3 files changed, 1002 insertions(+)
 create mode 100644 drive/MyDrive/Colab Notebooks/Laptop_price.csv
 create mode 100644 drive/MyDrive/Colab Notebooks/Pipeline.ipynb
 create mode 100644 drive/MyDrive/laptop_price_model.pkl


In [54]:
!git push -u origin master

Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (8/8), 94.87 KiB | 4.12 MiB/s, done.
Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/supertestuser1230/pipeline-model.git
 * [new branch]      master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.


In [58]:
!ngrok config add-authtoken 2Wd2eECXdnyQhQ2Ah9SbrKrGmw8_425Quw9aMfhvU5CSroiNw
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [59]:
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO
from pyngrok import ngrok

app = FastAPI()

model_path = "/content/drive/MyDrive/laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}

public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)


API доступно по адресу: NgrokTunnel: "https://678c-34-125-130-168.ngrok-free.app" -> "http://localhost:8000"


In [60]:
!git add /content/drive/MyDrive/Colab\ Notebooks/Pipeline.ipynb
!git commit -m "Добавлен FastAPI"
!git push -u origin master

[master 895d202] Добавлен FastAPI
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (6/6), 1.40 KiB | 1.40 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/supertestuser1230/pipeline-model.git
   416490a..895d202  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
